<a href="https://colab.research.google.com/github/springboardmentor789r/AgriYield/blob/Intern_LikhitaKoppuravuri/PreprocessedRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install category_encoders

In [1]:
#leave one out with min max scaling-lasso regression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
df = pd.read_excel("crop_yield_dataset.csv1.xlsm")
df = df.drop('Date', axis=1)
X = df.drop('Crop_Yield', axis=1)
y = df['Crop_Yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_df = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
target_col = 'Crop_Yield'
categorical_cols = ['Crop_Type', 'Soil_Type']
numerical_cols = ['Soil_pH', 'Temperature', 'Humidity', 'Wind_Speed', 'N', 'P', 'K', 'Soil_Quality']
def leave_one_out_encode(train_df, X_test, target_col, categorical_cols):
    global_mean = train_df[target_col].mean()
    X_train_encoded = X_train.copy().drop(categorical_cols, axis=1)
    X_test_encoded = X_test.copy().drop(categorical_cols, axis=1)

    for col in categorical_cols:
        agg = train_df.groupby(col)[target_col].agg(['count', 'sum'])
        N_i = agg['count']
        S_i = agg['sum']
        s_i_map = X_train[col].map(S_i)
        n_i_map = X_train[col].map(N_i)
        N_minus_1_count = n_i_map - 1
        S_minus_y = s_i_map - y_train
        looe_train = np.where(N_minus_1_count == 0, global_mean, S_minus_y / N_minus_1_count)
        X_train_encoded[f'{col}_Encoded'] = looe_train
        mean_map = (S_i / N_i).to_dict()
        X_test_encoded[f'{col}_Encoded'] = X_test[col].map(mean_map).fillna(global_mean)

    return X_train_encoded, X_test_encoded
X_train_encoded, X_test_encoded = leave_one_out_encode(train_df, X_test, target_col, categorical_cols)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_encoded.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_encoded.columns)
lasso_model = Lasso(alpha=1.0, random_state=42, max_iter=5000)
lasso_model.fit(X_train_scaled, y_train)
y_pred = lasso_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"Lasso Regression Model (LOOE + MinMax Scaling, alpha=1.0) Performance:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared ($R^2$): {r2:.4f}")
coefficients = pd.Series(lasso_model.coef_, index=X_train_scaled.columns)
print("\nModel Coefficients (Lasso, alpha=1.0):")
print(coefficients[coefficients != 0].sort_values(ascending=False))

Lasso Regression Model (LOOE + MinMax Scaling, alpha=1.0) Performance:
Mean Squared Error (MSE): 422.43
Root Mean Squared Error (RMSE): 20.55
R-squared ($R^2$): 0.1638

Model Coefficients (Lasso, alpha=1.0):
Soil_Type_Encoded    13.602969
Humidity             10.501559
dtype: float64


In [ ]:
#leave one out encoding with min max scaling-ridge regression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
df = pd.read_excel("crop_yield_dataset.csv1.xlsm")
df = df.drop('Date', axis=1)
X = df.drop('Crop_Yield', axis=1)
y = df['Crop_Yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_df = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
target_col = 'Crop_Yield'
categorical_cols = ['Crop_Type', 'Soil_Type']
numerical_cols = ['Soil_pH', 'Temperature', 'Humidity', 'Wind_Speed', 'N', 'P', 'K', 'Soil_Quality']
def leave_one_out_encode(train_df, X_test, target_col, categorical_cols):
    global_mean = train_df[target_col].mean()
    X_train_encoded = X_train.copy().drop(categorical_cols, axis=1)
    X_test_encoded = X_test.copy().drop(categorical_cols, axis=1)

    for col in categorical_cols:
        agg = train_df.groupby(col)[target_col].agg(['count', 'sum'])
        N_i = agg['count']
        S_i = agg['sum']
        s_i_map = X_train[col].map(S_i)
        n_i_map = X_train[col].map(N_i)
        N_minus_1_count = n_i_map - 1
        S_minus_y = s_i_map - y_train
        looe_train = np.where(N_minus_1_count == 0, global_mean, S_minus_y / N_minus_1_count)
        X_train_encoded[f'{col}_Encoded'] = looe_train
        mean_map = (S_i / N_i).to_dict()
        X_test_encoded[f'{col}_Encoded'] = X_test[col].map(mean_map).fillna(global_mean)

    return X_train_encoded, X_test_encoded
X_train_encoded, X_test_encoded = leave_one_out_encode(train_df, X_test, target_col, categorical_cols)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_encoded.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_encoded.columns)
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train_scaled, y_train)
y_pred = ridge_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"Ridge Regression Model (LOOE + MinMax Scaling, alpha=1.0) Performance:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared ($R^2$): {r2:.4f}")
coefficients = pd.Series(ridge_model.coef_, index=X_train_scaled.columns)
print("\nModel Coefficients (Ridge, alpha=1.0):")
print(coefficients.sort_values(ascending=False))

Ridge Regression Model (LOOE + MinMax Scaling, alpha=1.0) Performance:
Mean Squared Error (MSE): 214.75
Root Mean Squared Error (RMSE): 14.65
R-squared ($R^2$): 0.5749

Model Coefficients (Ridge, alpha=1.0):
Temperature          150.444116
Humidity             147.486490
Soil_Type_Encoded     14.587174
Crop_Type_Encoded      6.320599
Soil_Quality           5.956243
P                      4.154711
N                      0.987193
K                      0.454921
Soil_pH               -0.049528
Wind_Speed            -0.920407
dtype: float64


In [ ]:
#leave one out encoding with min max scaling-random forest regressor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
df = pd.read_excel("crop_yield_dataset.csv1.xlsm")
df = df.drop('Date', axis=1)
X = df.drop('Crop_Yield', axis=1)
y = df['Crop_Yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_df = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
target_col = 'Crop_Yield'
categorical_cols = ['Crop_Type', 'Soil_Type']
numerical_cols = ['Soil_pH', 'Temperature', 'Humidity', 'Wind_Speed', 'N', 'P', 'K', 'Soil_Quality']
def leave_one_out_encode(train_df, X_test, target_col, categorical_cols):
    global_mean = train_df[target_col].mean()
    X_train_encoded = X_train.copy().drop(categorical_cols, axis=1)
    X_test_encoded = X_test.copy().drop(categorical_cols, axis=1)

    for col in categorical_cols:
        agg = train_df.groupby(col)[target_col].agg(['count', 'sum'])
        N_i = agg['count']
        S_i = agg['sum']
        s_i_map = X_train[col].map(S_i)
        n_i_map = X_train[col].map(N_i)
        N_minus_1_count = n_i_map - 1
        S_minus_y = s_i_map - y_train
        looe_train = np.where(N_minus_1_count == 0, global_mean, S_minus_y / N_minus_1_count)
        X_train_encoded[f'{col}_Encoded'] = looe_train
        mean_map = (S_i / N_i).to_dict()
        X_test_encoded[f'{col}_Encoded'] = X_test[col].map(mean_map).fillna(global_mean)

    return X_train_encoded, X_test_encoded
X_train_encoded, X_test_encoded = leave_one_out_encode(train_df, X_test, target_col, categorical_cols)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_encoded.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_encoded.columns)
rf_regressor = RandomForestRegressor(random_state=42, n_jobs=-1)
rf_regressor.fit(X_train_scaled, y_train)
y_pred = rf_regressor.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"Random Forest Regressor (LOOE + MinMax Scaling) Performance:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared ($R^2$): {r2:.4f}")
importances = pd.Series(rf_regressor.feature_importances_, index=X_train_scaled.columns)
print("\nFeature Importances:")
print(importances.sort_values(ascending=False))

Random Forest Regressor (LOOE + MinMax Scaling) Performance:
Mean Squared Error (MSE): 267.93
Root Mean Squared Error (RMSE): 16.37
R-squared ($R^2$): 0.4697

Feature Importances:
Temperature          0.392911
Soil_Type_Encoded    0.289648
Soil_Quality         0.136310
Crop_Type_Encoded    0.078271
Humidity             0.045757
N                    0.025887
Soil_pH              0.020165
P                    0.009380
K                    0.001571
Wind_Speed           0.000099
dtype: float64


In [ ]:
#leave one out encoding with random forest regression with pipeline
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.metrics import mean_squared_error
df = pd.read_excel('crop_yield_dataset.csv1.xlsm')
X = df.drop('Crop_Yield', axis=1)
y = df['Crop_Yield']
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_cols),
    ('cat', LeaveOneOutEncoder(), categorical_cols)
])
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

param_grid = {
    'regressor__n_estimators': [100],
    'regressor__max_depth': [10, None],
    'regressor__min_samples_split': [2],
    'regressor__min_samples_leaf': [1]
}
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Test MSE: {mse:.4f}")
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best Parameters: {'regressor__max_depth': 10, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 100}
Test MSE: 253.4183
Best Parameters: {'regressor__max_depth': 10, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 100}
Mean Squared Error (MSE): 253.4183
Root Mean Squared Error (RMSE): 15.9191
Mean Absolute Error (MAE): 11.6821
R-squared (R²): 0.4984


In [ ]:
#leave one out encoding with min max scaler-random forest regression with pipeline
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
df = pd.read_excel('crop_yield_dataset.csv1.xlsm')
X = df.drop('Crop_Yield', axis=1)
y = df['Crop_Yield']
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
preprocessor = ColumnTransformer(transformers=[
    ('num', MinMaxScaler(), numerical_cols),
    ('cat', LeaveOneOutEncoder(), categorical_cols)
])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])
param_grid = {
    'regressor__n_estimators': [100],
    'regressor__max_depth': [10, None],
    'regressor__min_samples_split': [2, 5],
    'regressor__min_samples_leaf': [1, 2]
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Parameters: {'regressor__max_depth': 10, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 100}
Mean Squared Error (MSE): 253.1716
Root Mean Squared Error (RMSE): 15.9114
Mean Absolute Error (MAE): 11.6671
R-squared (R²): 0.4989


In [ ]:
#leave one out encoding with min max scaling-random forest without pipeline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
df = pd.read_excel('crop_yield_dataset.csv1.xlsm')
X = df.drop('Crop_Yield', axis=1)
y = df['Crop_Yield']
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
encoder = LeaveOneOutEncoder(cols=categorical_cols)
X_train_cat_encoded = encoder.fit_transform(X_train[categorical_cols], y_train)
X_test_cat_encoded = encoder.transform(X_test[categorical_cols])
scaler = MinMaxScaler()
X_train_num_scaled = scaler.fit_transform(X_train[numerical_cols])
X_test_num_scaled = scaler.transform(X_test[numerical_cols])
X_train_processed = np.hstack((X_train_num_scaled, X_train_cat_encoded))
X_test_processed = np.hstack((X_test_num_scaled, X_test_cat_encoded))
rf = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [100],
    'max_depth': [10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train_processed, y_train)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_processed)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Mean Squared Error (MSE): 267.8733
Root Mean Squared Error (RMSE): 16.3668
Mean Absolute Error (MAE): 12.3133
R-squared (R²): 0.4698


In [ ]:
print(y_train.head())
print(y_train.isna().sum())
print(y_train.dtype)



13222    27.916667
3295     37.041667
4401     14.583333
2310     22.500000
15429    13.916667
Name: Soil_Quality, dtype: float64
0
float64


In [ ]:
#leave one out encoding with linear regression
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
df = pd.read_excel("crop_yield_dataset.csv1.xlsm")
df_processed = df.drop('Date', axis=1)
X = df_processed.drop('Crop_Yield', axis=1)
y = df_processed['Crop_Yield']
full_df = df_processed.copy()
categorical_cols = ['Crop_Type', 'Soil_Type']
target_col = 'Crop_Yield'
def leave_one_out_encode_full(df, target_col, categorical_cols):
    df_encoded = df.drop(target_col, axis=1).copy()
    global_mean = df[target_col].mean()

    for col in categorical_cols:
        agg = df.groupby(col)[target_col].agg(['count', 'sum'])
        N_i = agg['count']
        S_i = agg['sum']
        df_encoded[f'{col}_sum'] = df_encoded[col].map(S_i)
        df_encoded[f'{col}_count'] = df_encoded[col].map(N_i)
        N_minus_1 = df_encoded[f'{col}_count'] - 1
        S_minus_y = df_encoded[f'{col}_sum'] - df[target_col]
        looe = np.where(N_minus_1 == 0, global_mean, S_minus_y / N_minus_1)
        df_encoded[f'{col}_Encoded'] = looe
        df_encoded = df_encoded.drop([col, f'{col}_sum', f'{col}_count'], axis=1)

    return df_encoded
X_encoded = leave_one_out_encode_full(full_df, target_col, categorical_cols)
linear_model = LinearRegression()
linear_model.fit(X_encoded, y)
y_pred_full = linear_model.predict(X_encoded)
mse_full = mean_squared_error(y, y_pred_full)
rmse_full = np.sqrt(mse_full)
r2_full = r2_score(y, y_pred_full)
print(f"--- Results on the Complete Dataset (In-Sample Performance) ---")
print(f"Mean Squared Error (MSE): {mse_full:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse_full:.2f}")
print(f"R-squared ($R^2$): {r2_full:.4f}")
coefficients_full = pd.Series(linear_model.coef_, index=X_encoded.columns)
print("\nModel Coefficients:")
print(coefficients_full)

--- Results on the Complete Dataset (In-Sample Performance) ---
Mean Squared Error (MSE): 217.20
Root Mean Squared Error (RMSE): 14.74
R-squared ($R^2$): 0.5670

Model Coefficients:
Soil_pH             -0.023937
Temperature          6.090198
Humidity             9.949944
Wind_Speed          -0.014758
N                    0.003710
P                    0.186517
K                   -0.013511
Soil_Quality         0.105804
Crop_Type_Encoded    0.618596
Soil_Type_Encoded    0.659791
dtype: float64


In [ ]:
#leave one out encoding with standard scaling-logistic regression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import numpy as np
df = pd.read_excel("crop_yield_dataset.csv1.xlsm")
df = df.drop('Date', axis=1)
median_yield = df['Crop_Yield'].median()
df['High_Yield'] = (df['Crop_Yield'] > median_yield).astype(int)
X = df.drop(['Crop_Yield', 'High_Yield'], axis=1)
y = df['High_Yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
train_df = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
target_col = 'High_Yield'
categorical_cols = ['Crop_Type', 'Soil_Type']
numerical_cols = ['Soil_pH', 'Temperature', 'Humidity', 'Wind_Speed', 'N', 'P', 'K', 'Soil_Quality']
def leave_one_out_encode(train_df, X_test, target_col, categorical_cols):
    # Calculate the global mean of the binary target in the training set
    global_mean = train_df[target_col].mean()
    X_train_encoded = X_train.copy().drop(categorical_cols, axis=1)
    X_test_encoded = X_test.copy().drop(categorical_cols, axis=1)

    for col in categorical_cols:
        agg = train_df.groupby(col)[target_col].agg(['count', 'sum'])
        N_i = agg['count']
        S_i = agg['sum']
        s_i_map = X_train[col].map(S_i)
        n_i_map = X_train[col].map(N_i)
        N_minus_1 = n_i_map - y_train # Here, y_train is 0 or 1, but we need n_i_map - 1 for count
        N_minus_1_count = n_i_map - 1
        S_minus_y = s_i_map - y_train
        looe_train = np.where(N_minus_1_count == 0, global_mean, S_minus_y / N_minus_1_count)
        X_train_encoded[f'{col}_Encoded'] = looe_train
        mean_map = (S_i / N_i).to_dict()
        X_test_encoded[f'{col}_Encoded'] = X_test[col].map(mean_map).fillna(global_mean)

    return X_train_encoded, X_test_encoded
X_train_encoded, X_test_encoded = leave_one_out_encode(train_df, X_test, target_col, categorical_cols)
scaler = StandardScaler()
X_train_encoded[numerical_cols] = scaler.fit_transform(X_train_encoded[numerical_cols])
X_test_encoded[numerical_cols] = scaler.transform(X_test_encoded[numerical_cols])
logreg = LogisticRegression(random_state=42, max_iter=1000)
logreg.fit(X_train_encoded, y_train)
y_pred = logreg.predict(X_test_encoded)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Logistic Regression Model (LOOE + Scaling) Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(report)

Logistic Regression Model (LOOE + Scaling) Accuracy: 0.7953

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.78      0.79      2550
           1       0.79      0.81      0.80      2549

    accuracy                           0.80      5099
   macro avg       0.80      0.80      0.80      5099
weighted avg       0.80      0.80      0.80      5099



In [ ]:
#leave one out encdoing with standard scaling-decision tree regression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np
df = pd.read_excel("crop_yield_dataset.csv1.xlsm")
df = df.drop('Date', axis=1)
X = df.drop('Crop_Yield', axis=1)
y = df['Crop_Yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_df = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
target_col = 'Crop_Yield'
categorical_cols = ['Crop_Type', 'Soil_Type']
numerical_cols = ['Soil_pH', 'Temperature', 'Humidity', 'Wind_Speed', 'N', 'P', 'K', 'Soil_Quality']
def leave_one_out_encode(train_df, X_test, target_col, categorical_cols):
    global_mean = train_df[target_col].mean()
    X_train_encoded = X_train.copy().drop(categorical_cols, axis=1)
    X_test_encoded = X_test.copy().drop(categorical_cols, axis=1)

    for col in categorical_cols:
        agg = train_df.groupby(col)[target_col].agg(['count', 'sum'])
        N_i = agg['count']
        S_i = agg['sum']
        s_i_map = X_train[col].map(S_i)
        n_i_map = X_train[col].map(N_i)
        N_minus_1_count = n_i_map - 1
        S_minus_y = s_i_map - y_train
        looe_train = np.where(N_minus_1_count == 0, global_mean, S_minus_y / N_minus_1_count)
        X_train_encoded[f'{col}_Encoded'] = looe_train
        mean_map = (S_i / N_i).to_dict()
        X_test_encoded[f'{col}_Encoded'] = X_test[col].map(mean_map).fillna(global_mean)

    return X_train_encoded, X_test_encoded
X_train_encoded, X_test_encoded = leave_one_out_encode(train_df, X_test, target_col, categorical_cols)
scaler = StandardScaler()
X_train_encoded[numerical_cols] = scaler.fit_transform(X_train_encoded[numerical_cols])
X_test_encoded[numerical_cols] = scaler.transform(X_test_encoded[numerical_cols])
dt_regressor = DecisionTreeRegressor(random_state=42, max_depth=10)
dt_regressor.fit(X_train_encoded, y_train)
y_pred = dt_regressor.predict(X_test_encoded)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"Decision Tree Regressor (LOOE + Scaling, max_depth=10) Performance:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared ($R^2$): {r2:.4f}")

Decision Tree Regressor (LOOE + Scaling, max_depth=10) Performance:
Mean Squared Error (MSE): 267.62
Root Mean Squared Error (RMSE): 16.36
R-squared ($R^2$): 0.4703


In [ ]:
#leave one out encoding with standard scaling-support vector regression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np
df = pd.read_excel("crop_yield_dataset.csv1.xlsm")
df = df.drop('Date', axis=1)
X = df.drop('Crop_Yield', axis=1)
y = df['Crop_Yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_df = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
target_col = 'Crop_Yield'
categorical_cols = ['Crop_Type', 'Soil_Type']
numerical_cols = ['Soil_pH', 'Temperature', 'Humidity', 'Wind_Speed', 'N', 'P', 'K', 'Soil_Quality']
def leave_one_out_encode(train_df, X_test, target_col, categorical_cols):
    global_mean = train_df[target_col].mean()
    X_train_encoded = X_train.copy().drop(categorical_cols, axis=1)
    X_test_encoded = X_test.copy().drop(categorical_cols, axis=1)

    for col in categorical_cols:
        agg = train_df.groupby(col)[target_col].agg(['count', 'sum'])
        N_i = agg['count']
        S_i = agg['sum']
        s_i_map = X_train[col].map(S_i)
        n_i_map = X_train[col].map(N_i)
        N_minus_1_count = n_i_map - 1
        S_minus_y = s_i_map - y_train

        looe_train = np.where(N_minus_1_count == 0, global_mean, S_minus_y / N_minus_1_count)
        X_train_encoded[f'{col}_Encoded'] = looe_train
        mean_map = (S_i / N_i).to_dict()
        X_test_encoded[f'{col}_Encoded'] = X_test[col].map(mean_map).fillna(global_mean)

    return X_train_encoded, X_test_encoded
X_train_encoded, X_test_encoded = leave_one_out_encode(train_df, X_test, target_col, categorical_cols)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_encoded.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_encoded.columns)
svr_model = LinearSVR(random_state=42, max_iter=5000, dual=False, loss='squared_epsilon_insensitive')
svr_model.fit(X_train_scaled, y_train)
y_pred = svr_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"Linear Support Vector Regressor (LOOE + Scaling) Performance:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared ($R^2$): {r2:.4f}")

Linear Support Vector Regressor (LOOE + Scaling) Performance:
Mean Squared Error (MSE): 214.72
Root Mean Squared Error (RMSE): 14.65
R-squared ($R^2$): 0.5750


In [ ]:
#leave one out encoding with min max scaling -linear regression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
df = pd.read_excel("crop_yield_dataset.csv1.xlsm")
df = df.drop('Date', axis=1)
X = df.drop('Crop_Yield', axis=1)
y = df['Crop_Yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_df = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
target_col = 'Crop_Yield'
categorical_cols = ['Crop_Type', 'Soil_Type']
numerical_cols = ['Soil_pH', 'Temperature', 'Humidity', 'Wind_Speed', 'N', 'P', 'K', 'Soil_Quality']
def leave_one_out_encode(train_df, X_test, target_col, categorical_cols):
    global_mean = train_df[target_col].mean()
    X_train_encoded = X_train.copy().drop(categorical_cols, axis=1)
    X_test_encoded = X_test.copy().drop(categorical_cols, axis=1)

    for col in categorical_cols:
        agg = train_df.groupby(col)[target_col].agg(['count', 'sum'])
        N_i = agg['count']
        S_i = agg['sum']
        s_i_map = X_train[col].map(S_i)
        n_i_map = X_train[col].map(N_i)
        N_minus_1_count = n_i_map - 1
        S_minus_y = s_i_map - y_train
        looe_train = np.where(N_minus_1_count == 0, global_mean, S_minus_y / N_minus_1_count)
        X_train_encoded[f'{col}_Encoded'] = looe_train
        mean_map = (S_i / N_i).to_dict()
        X_test_encoded[f'{col}_Encoded'] = X_test[col].map(mean_map).fillna(global_mean)

    return X_train_encoded, X_test_encoded
X_train_encoded, X_test_encoded = leave_one_out_encode(train_df, X_test, target_col, categorical_cols)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_encoded.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_encoded.columns)
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)
y_pred = linear_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"Linear Regression Model (LOOE + MinMax Scaling) Performance:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared ($R^2$): {r2:.4f}")
coefficients = pd.Series(linear_model.coef_, index=X_train_scaled.columns)
print("\nModel Coefficients (after MinMax Scaling):")
print(coefficients.sort_values(ascending=False))

Linear Regression Model (LOOE + MinMax Scaling) Performance:
Mean Squared Error (MSE): 214.72
Root Mean Squared Error (RMSE): 14.65
R-squared ($R^2$): 0.5750

Model Coefficients (after MinMax Scaling):
Temperature          152.243177
Humidity             149.034351
Soil_Type_Encoded     14.770173
Crop_Type_Encoded      6.403987
Soil_Quality           5.782022
P                      4.119808
N                      0.945118
K                      0.390199
Soil_pH               -0.056447
Wind_Speed            -0.918339
dtype: float64


In [ ]:
#one hot encoding standard scaling-decision tree regression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np
df = pd.read_excel("crop_yield_dataset.csv1.xlsm")
df = df.drop('Date', axis=1)
X = df.drop('Crop_Yield', axis=1)
y = df['Crop_Yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
categorical_cols = ['Crop_Type', 'Soil_Type']
numerical_cols = ['Soil_pH', 'Temperature', 'Humidity', 'Wind_Speed', 'N', 'P', 'K', 'Soil_Quality']
X_train_encoded = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)
train_cols = X_train_encoded.columns
X_test_encoded = X_test_encoded.reindex(columns=train_cols, fill_value=0)
scaler = StandardScaler()
scaler.fit(X_train_encoded[numerical_cols])

X_train_encoded[numerical_cols] = scaler.transform(X_train_encoded[numerical_cols])
X_test_encoded[numerical_cols] = scaler.transform(X_test_encoded[numerical_cols])
dt_regressor = DecisionTreeRegressor(random_state=42, max_depth=10)
dt_regressor.fit(X_train_encoded, y_train)
y_pred = dt_regressor.predict(X_test_encoded)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"Decision Tree Regressor (max_depth=10) Performance:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared ($R^2$): {r2:.4f}")

Decision Tree Regressor (max_depth=10) Performance:
Mean Squared Error (MSE): 29.42
Root Mean Squared Error (RMSE): 5.42
R-squared ($R^2$): 0.9418


In [ ]:
#one hot encoding with standard scaling-logisteic regression
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import numpy as np
df = pd.read_csv("crop_yield_dataset.csv")
df = df.drop('Date', axis=1)
median_yield = df['Crop_Yield'].median()
df['High_Yield'] = (df['Crop_Yield'] > median_yield).astype(int)
X = df.drop(['Crop_Yield', 'High_Yield'], axis=1)
y = df['High_Yield']
categorical_cols = ['Crop_Type', 'Soil_Type']
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
numerical_cols = ['Soil_pH', 'Temperature', 'Humidity', 'Wind_Speed', 'N', 'P', 'K', 'Soil_Quality']
scaler = StandardScaler()
X_scaled = X_encoded.copy()
X_scaled[numerical_cols] = scaler.fit_transform(X_encoded[numerical_cols])
logreg = LogisticRegression(random_state=42, max_iter=1000)
logreg.fit(X_scaled, y)
y_pred_full = logreg.predict(X_scaled)
accuracy_full = accuracy_score(y, y_pred_full)
report_full = classification_report(y, y_pred_full)
print(f"--- Results on the Complete Dataset (In-Sample Performance) ---")
print(f"Logistic Regression Model Accuracy: {accuracy_full:.4f}")
print("\nClassification Report:")
print(report_full)
coefficients_full = pd.Series(logreg.coef_[0], index=X_scaled.columns)
print("\nModel Coefficients:")
print(coefficients_full.sort_values(ascending=False))

In [ ]:
import pandas as pd
from google.colab import files
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
import numpy as np
import sys

# --- Configuration ---
# ✅ CORRECTED FILE NAME
FILE_NAME = "crop_yield_dataset.csv1.xlsm"
SHEET_NAME = 'crop_yield_dataset (1)'
TARGET_COL = 'Crop_Yield'
CAT_COLS = ['Crop_Type', 'Soil_Type']
# All features the model expects:
FEATURE_COLS = ['Soil_pH', 'Temperature', 'Humidity', 'Wind_Speed', 'N', 'P', 'K', 'Soil_Quality', 'Crop_Type', 'Soil_Type']


# --- 1. Training Block: Load, Preprocess, and Fit the Model ---

def train_lasso_model():
    """Loads data, performs preprocessing (encoding/scaling), and fits the Lasso model."""

    try:
        # Load the XLSM file
        print(f"1. Loading '{FILE_NAME}'...")
        # Use read_excel for the .xlsm file
        df = pd.read_excel(FILE_NAME = "crop_yield_dataset.csv1.xlsm", sheet_name='crop_yield_dataset (1)', engine='openpyxl')

        y = df[TARGET_COL]
        # X includes all features, dropping 'Date'
        X = df.drop(columns=[TARGET_COL, 'Date'], errors='ignore')

    except Exception as e:
        print(f"\nFATAL ERROR: Could not load the Excel file.")
        print("1. Ensure the file is uploaded to Colab.")
        print(f"2. Check if the SHEET_NAME ('{SHEET_NAME}') is exactly correct.")
        print(f"Error details: {e}")
        sys.exit(1)

    # --- 1a. Leave One Out Encoding (LOOE) Substitute: Target Mean Encoding ---
    # Store the means for use in the prediction step
    fitted_target_means = {}

    print("2. Performing Target Mean Encoding (LOOE Substitute)...")
    for col in CAT_COLS:
        if col in X.columns:
            # Calculate the mean of the target 'y' for each category
            target_means = y.groupby(X[col]).mean()
            fitted_target_means[col] = target_means
            # Replace the categorical values with the calculated mean
            X[col] = X[col].map(target_means)
        else:
            # This should only happen if your file structure is missing columns
            print(f"   Warning: Column '{col}' not found. Skipping encoding.")

    # --- 1b. Min Max Scaling ---
    print("3. Performing Min Max Scaling on all features...")

    fitted_scaler = MinMaxScaler()

    # Fit the scaler to ALL feature columns
    X_scaled_array = fitted_scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled_array, columns=X.columns)

    print("   Scaling complete.")

    # --- 1c. Lasso Regression Fit ---
    print("4. Fitting Lasso Regression Model...")
    lasso = Lasso(alpha=0.1, random_state=42)
    lasso.fit(X_scaled, y)
    print("   Model training complete.")

    # Return all fitted objects needed for prediction
    return lasso, fitted_scaler, fitted_target_means, X_scaled.columns.tolist()


# --- 2. Prediction Block: Use Fitted Objects to Predict ---

def predict_crop_yield(lasso_model, scaler, target_means, feature_order):
    """
    Predicts Crop Yield for a single row of input parameters.

    Replace the values in 'new_input_data' with your desired parameters.
    """
    print("\n\n--- PREDICTION ---")

    # Define your input parameters (Features)
    # 🌟🌟🌟 CHANGE THESE VALUES FOR YOUR PREDICTION 🌟🌟🌟
    new_input_data = {
        'Soil_pH': 6.5,
        'Temperature': 25.0,
        'Humidity': 80.0,
        'Wind_Speed': 10.0,
        'N': 90.0,
        'P': 60.0,
        'K': 50.0,
        'Soil_Quality': 50.0,
        'Crop_Type': 'Corn',
        'Soil_Type': 'Loamy'
    }

    # Convert the new input into a DataFrame
    new_df = pd.DataFrame([new_input_data])

    # Ensure the columns are in the exact order the model was trained on
    new_df = new_df[feature_order]

    # 1. Apply Encoding
    for col, means in target_means.items():
        # Map the categorical value to its corresponding mean from training data
        new_df[col] = new_df[col].map(means)
        # Fallback for new/unknown categories (uses the average of all means)
        if new_df[col].isnull().any():
            new_df[col] = new_df[col].fillna(np.mean(list(means.values())))

    # 2. Apply Scaling
    new_df_scaled = scaler.transform(new_df)

    # 3. Use the predict() function
    predicted_yield = lasso_model.predict(new_df_scaled)

    # 4. Display the result
    print("\n#################################")
    print(f"The Predicted Crop Yield is: {predicted_yield[0]:.2f}")
    print("#################################")

# --- Execute ---

# 1. Train the model and get the fitted components
lasso_model, fitted_scaler, fitted_target_means, feature_order = train_lasso_model()

# 2. Perform the prediction
predict_crop_yield(lasso_model, fitted_scaler, fitted_target_means, feature_order)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



1. Loading 'crop_yield_dataset.csv1.xlsm'...

FATAL ERROR: Could not load the Excel file.
1. Ensure the file is uploaded to Colab.
2. Check if the SHEET_NAME ('crop_yield_dataset (1)') is exactly correct.
Error details: read_excel() got an unexpected keyword argument 'FILE_NAME'
Traceback (most recent call last):
  File "/tmp/ipython-input-1169179829.py", line 27, in train_lasso_model
    df = pd.read_excel(FILE_NAME = "crop_yield_dataset.csv1.xlsm", sheet_name='crop_yield_dataset (1)', engine='openpyxl')
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: read_excel() got an unexpected keyword argument 'FILE_NAME'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-116

TypeError: object of type 'NoneType' has no len()